## Свёрточные сети для классификации

In [2]:
from typing import Type

import torch
from torch import Tensor, nn
from torch.nn import functional as F

#### Задание 1. Skip-connections (2 балла)

Постройте архитектуру свёрточной сети, аналогичную архитектуре в примере ниже, но добавьте в неё skip-connections, то есть дополнительные рёбра в вычислительном графе, позволяющие пропускать градиент в более ранние слои напрямую, минуя очередной блок Conv2D + BatchNorm + ReLU:

```python
def forward(self, x: Tensor) -> Tensor:
    x = x + self.block1(x)
    x = self.maxpool(x)
    x = x + self.block2(x)
    x = self.maxpool(x)
    ...
    x = x.adaptive_maxpool(x).flatten(1)
    logits = self.fc(x)
    return logits
```


Наша верхнеуровневая архитектура будет выглядеть так:

In [3]:
class MyResNet(nn.Module):
    def __init__(
        self,
        block: Type[nn.Module],
        n_classes: int,
        hidden_channels: list[int] = [32, 64],
    ) -> None:
        super().__init__()
        # входной слой, принимающий изображение с 3-мя каналами
        self.in_conv = nn.Conv2d(3, hidden_channels[0], kernel_size=3, stride=1)
        self.relu = nn.ReLU(inplace=True)

        # собираем свёрточные блоки, каждый задаётся кол-вом входных и выходных каналов
        blocks = []
        for c_in, c_out in zip(hidden_channels[:-1], hidden_channels[1:]):
            # добавляем очередной блок
            blocks.append(block(c_in, c_out))
            # добавляем Max pooling для уменьшения размерности
            blocks.append(nn.MaxPool2d(2, 2))

        # собираем блоки в единый Sequential модуль для удобства
        self.features = nn.Sequential(*blocks)
        self.maxpool = nn.AdaptiveMaxPool2d(1)

        # линейный слой для классификации
        self.fc = nn.Linear(hidden_channels[-1], n_classes)

    def forward(self, x: Tensor) -> Tensor:
        h = self.features(self.relu(self.in_conv(x)))
        logits = self.fc(self.maxpool(h).flatten(1))
        return logits

Базовый блок, без residual connections, состоит из двух свёрток и нормализаций:

In [4]:
class BasicBlock(nn.Module):
    def __init__(self, inplanes: int, planes: int) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(
            inplanes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

    def forward(self, x: Tensor) -> Tensor:
        # first conv + bn + nonlinearity
        out = self.relu(self.bn1(self.conv1(x)))
        # second conv + bn
        out = self.bn2(self.conv2(out))
        # final nonlinearity
        out = self.relu(out)
        return out

Посмотрим на результат его применения к тензору:

In [5]:
BasicBlock(4, 6).forward(torch.randn(3, 4, 32, 32)).shape

torch.Size([3, 6, 32, 32])

Теперь нужно изменить этот блок, добавив в него skip-connection. Теперь в методе `forward` входной тензор `x` пойдёт по двум веткам:
1. как в базовом блоке, через наши всёртки и нормализации, до последней нелинейности
2. в обход свёрток и нормализаций

В конце эти ветки нужно объединить через сумму. Тут есть проблема: в исходном тензоре `x` и обработанном нашим блоком `h(x)` отличается количество каналов (остальные размерности совпадают). То есть нам нужно сравнять количество каналов исходного тензора `inplanes` с количеством выходных каналов `outplanes`.

Интуитивно, если рассматривать каждый пиксел входного тензора как вектор размера `inplanes`, в вектор размера `planes` его можно превратить домножением на матрицу размера `inplanes x planes`. Это можно сделать, создав свёрточный слой с размером кернела 1 - он и будет переводить наши пикселы в другую размерность.

Не забудьте к сумме каналов применить нелинейность.

In [6]:
class ResidualBlock(nn.Module):
    def __init__(self, inplanes: int, planes: int) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(
            inplanes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            planes, planes, kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(planes)

        # добавьте свёртку 1x1 для изменения кол-ва каналов входного тензора

        self.conv = nn.Conv2d(inplanes, planes, kernel_size=1, bias=False)
    def forward(self, x: Tensor) -> Tensor:
        # сохраним входной тензор на будущее
        identity = x

        # ВАШ ХОД
        out = self.bn2(self.conv2(self.relu(self.bn1(self.conv1(x)))))
        out += self.conv(identity)
        return self.relu(out)

Проверим размеры:

In [7]:
assert ResidualBlock(4, 6).forward(torch.randn(3, 4, 32, 32)).shape == torch.Size(
    [3, 6, 32, 32]
)

Проверим, что модель выдаёт тензор ожидаемого размера:

In [8]:
MyResNet(ResidualBlock, 7, hidden_channels=[16, 32, 64, 128]).forward(
    torch.randn(3, 3, 32, 32)
).shape

torch.Size([3, 7])

Теперь мы можем создавать модели разного размера, в том числе достаточно большие и глубокие, чтобы хорошо классифицировать изображения из датасета CIFAR-10.

In [9]:
sum(
    p.numel()
    for p in MyResNet(ResidualBlock, 7, hidden_channels=[16, 32, 64, 64]).parameters()
)

151047

#### Задание 2. Обучение `MyResNet` с использованием Lightning (5 баллов)

Ваша задача: добиться 80% точности на валидационной выборке с вашей реализацией `MyResNet`.

После окончания обучения используйте метод `Trainer.validate` для вывода ваших метрик с удачного чекпоинта модели.

NB: вызывайте `Trainer.validate` везде, где в задании требуется достичь какой-то точности


Советы:
- По умолчанию Lightning сохраняет только последний чекпоинт, так что вам может потребоваться `lightning.callbacks.ModelCheckpoint`, чтобы сохранять лучший чекпоинт в процессе обучения.

- Используйте tensorboard, чтобы следить за динамикой обучения. Если заметите переобучение - подключайте регуляризацию. Большая модель с регуляризацией обычно лучше маленькой модели без неё.

- Чтобы добиться нужной точности, ваша модель должна быть достаточно глубокой, ориентируйтесь на 4-5 блоков. Если необходимо, подключайте регуляризацию

In [10]:
!pip install lightning
!pip install torchmetrics

from typing import Callable
import lightning as L
import torch
import torchmetrics
from torch import Tensor, nn
from torch.nn import functional as F
from lightning.pytorch.utilities.types import EVAL_DATALOADERS, TRAIN_DATALOADERS
from PIL.Image import Image
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


class Datamodule(L.LightningDataModule):
    def __init__(
        self,
        batch_size: int,
        transform: Callable[[Image], Tensor] = transforms.ToTensor(),
        num_workers: int = 0,
    ):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transform
        self.num_workers = num_workers

    def prepare_data(self) -> None:
        # в этом методе можно сделать предварительную работу, например
        # скачать данные, сделать тяжёлый препроцессинг
        pass

    def setup(self, stage: str) -> None:
        # аргумент `stage` будет приходить из модуля обучения Trainer
        # на стадии обучения (fit) нам нужны оба датасета
        if stage == "fit":
            self.train_dataset = datasets.CIFAR10(
                "data",
                train=True,
                download=True,
                transform=self.transform,
            )
            self.val_dataset = datasets.CIFAR10(
                "data",
                train=False,
                download=True,
                transform=transforms.ToTensor(),
            )
        # на стадии валидации (validate) - только тестовый
        elif stage == "validate":
            self.val_dataset = datasets.CIFAR10(
                "data",
                train=False,
                download=True,
                transform=self.transform,
            )
        else:
            raise NotImplementedError
        # есть ещё стадии `test` и `predict`, но они нам не понадобятся

    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

    def val_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 869.5/869.5 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 52.9 MB/s eta 0:00:00


In [15]:
datamodule = Datamodule(batch_size=256, num_workers=0)
datamodule.setup(stage="fit")
batch = next(iter(datamodule.train_dataloader()))
for item in batch:
    print(item.shape)

Files already downloaded and verified
Files already downloaded and verified
torch.Size([256, 3, 32, 32])
torch.Size([256])


In [16]:
import torchmetrics.classification
from typing import Any

from lightning.pytorch.utilities.types import STEP_OUTPUT
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint



def create_classification_metrics(
    num_classes: int, prefix: str
) -> torchmetrics.MetricCollection:
    return torchmetrics.MetricCollection(
        [
            torchmetrics.Accuracy(task="multiclass", num_classes=num_classes),
            torchmetrics.classification.MulticlassAUROC(
                num_classes=num_classes, average="macro"
            ),
        ],
        prefix=prefix,
    )


class Lit(L.LightningModule):
    def __init__(self, model: nn.Module, learning_rate: float) -> None:
        super().__init__()
        self.save_hyperparameters()
        self.model = model
        self.learning_rate = learning_rate
        self.train_metrics = create_classification_metrics(
            num_classes=10, prefix="train_"
        )
        self.val_metrics = create_classification_metrics(num_classes=10, prefix="val_")

    def training_step(
        self, batch: tuple[Tensor, Tensor], batch_idx: int
    ) -> STEP_OUTPUT:
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        # loss теперь сохраняем только раз в эпоху
        self.log("train_loss", loss, on_epoch=True, on_step=False)
        # обновляем метрики и логируем раз в эпоху
        self.train_metrics.update(y_hat, y)
        self.log_dict(self.train_metrics, on_step=False, on_epoch=True)

        return loss

    def validation_step(
        self, batch: tuple[Tensor, Tensor], batch_idx: int
    ) -> STEP_OUTPUT | None:
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        self.log("val_loss", loss, on_epoch=True, on_step=False)
        # обновляем метрики и логируем раз в эпоху
        self.val_metrics.update(y_hat, y)
        self.log_dict(self.val_metrics, on_step=False, on_epoch=True)
        # на этот раз вернём предсказания - будем их потом использовать, чтобы отрисовывать confusion matrix

        return {
            "loss": loss,
            "preds": y_hat,
        }

    def on_validation_epoch_end(self) -> None:
        val_loss = self.trainer.callback_metrics["val_loss"]
        val_accuracy = self.trainer.callback_metrics["val_MulticlassAccuracy"]
        print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
        self.val_metrics.reset()

    def configure_optimizers(self) -> dict[str, Any]:
        optimizer = torch.optim.Adam(self.model.parameters(), lr=self.learning_rate, weight_decay=1e-4)
        # давайте кроме оптимизатора создадим ещё расписание для шага оптимизации
        return {
            "optimizer": optimizer,
            "lr_scheduler": torch.optim.lr_scheduler.MultiStepLR(
                optimizer, milestones=[5, 10, 15]
            ),
        }

In [17]:
trainer = L.Trainer(
    accelerator="auto",
    max_epochs=15,
    limit_train_batches=100,
    limit_val_batches=100,
    logger=TensorBoardLogger(save_dir="."),
)
lit_module = Lit(
    model= MyResNet(ResidualBlock, 10, hidden_channels=[32, 64, 128, 256]),
    learning_rate=0.0007952812,
)
trainer.fit(
    model=lit_module,
    datamodule=datamodule,
)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'model' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['model'])`.


Files already downloaded and verified
Files already downloaded and verified


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | MyResNet         | 1.2 M  | train
1 | train_metrics | MetricCollection | 0      | train
2 | val_metrics   | MetricCollection | 0      | train
-----------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.838     Total estimated model params size (MB)
36        Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | MyResNet         | 1.2 M  | train
1 | train_metrics | MetricCollection | 0      | train
2 | val_metrics   | MetricCollection | 0      | train
---------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Validation Loss: 2.3038, Validation Accuracy: 0.1191


...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation Loss: 0.4305, Validation Accuracy: 0.8518


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=15` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=15` reached.


Validation Loss: 0.4293, Validation Accuracy: 0.8523


In [18]:
trainer.validate(model=lit_module, dataloaders=datamodule.val_dataloader())

Files already downloaded and verified


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

Validation Loss: 0.4293, Validation Accuracy: 0.8523


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    val_MulticlassAUROC    │    0.9875971674919128     │
│  val_MulticlassAccuracy   │    0.8522999882698059     │
│         val_loss          │    0.42927470803260803    │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.42927470803260803,
  'val_MulticlassAccuracy': 0.8522999882698059,
  'val_MulticlassAUROC': 0.9875971674919128}]

#### Задание 3. Добавление аугментаций (1 балл + 2 балла за точность на валидации более 85%)

Добавьте к обучающему датасету аугментации - случайные трансформации входных данных. Для этого можно использовать `torchvision.transforms` и `albumentations`.

С `torchvision.transforms` совсем просто: вам нужно будет при создании `Datamodule` из практики по `lightning` указать вместо

```python
transform = transforms.ToTensor()
```
композицию трансформаций:

```python
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # случайное зеркальное отражение
    ...
    transforms.ToTensor(),
])
```

In [30]:
!pip install -U albumentations


В пакете `albumentations` аугментаций значительно больше:

![albumentations](https://albumentations.ai/assets/img/custom/top_image.jpg)

In [48]:
import albumentations as A
import torchvision.transforms as tt

datamodule_albumentations = Datamodule(batch_size=256, num_workers=0, transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # случайное зеркальное отражение
    transforms.GaussianBlur(1),
    transforms.RandomAdjustSharpness(0.5),
    transforms.ToTensor()
]))

datamodule_albumentations.setup(stage="fit")

trainer = L.Trainer(
    accelerator="auto",
    max_epochs=20,
    limit_train_batches=100,
    limit_val_batches=100,
    logger=TensorBoardLogger(save_dir="."),
)
lit_module = Lit(
    model= MyResNet(ResidualBlock, 10, hidden_channels=[32, 64, 128, 256]),
    learning_rate=0.0007952812,
)
trainer.fit(
    model=lit_module,
    datamodule=datamodule_albumentations,
)

Files already downloaded and verified
Files already downloaded and verified


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | MyResNet         | 1.2 M  | train
1 | train_metrics | MetricCollection | 0      | train
2 | val_metrics   | MetricCollection | 0      | train
-----------------------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.838     Total estimated model params size (MB)
36        Modules in train mode
0         Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | MyResNet         | 1.2 M  | train
1 | train_metrics | MetricCollection | 0      | train
2 | val_metrics   | MetricCollection | 0      | train
---------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation Loss: 0.4335, Validation Accuracy: 0.8561


Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=20` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


Validation Loss: 0.4330, Validation Accuracy: 0.8565


In [49]:
trainer.validate(model=lit_module, dataloaders=datamodule.val_dataloader())

Files already downloaded and verified


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: |          | 0/? [00:00<?, ?it/s]

Validation Loss: 0.4330, Validation Accuracy: 0.8565


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│    val_MulticlassAUROC    │    0.9873026013374329     │
│  val_MulticlassAccuracy   │    0.8565000295639038     │
│         val_loss          │     0.432997465133667     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 0.432997465133667,
  'val_MulticlassAccuracy': 0.8565000295639038,
  'val_MulticlassAUROC': 0.9873026013374329}]

#### Задание 4. Использование предобученной модели (4 балла)

Теперь мы научимся использовать модели, обученные на других задачах

Ваша задача: добиться 90% точности на тестовой выборке CIFAR-10. Постарайтесь уложиться модель с ~5 млн параметров

В `torchvision.models` есть много реализованных архитектур, размером которых можно удобно управлять. Например, ниже можно создать крошечную версию модели `MobileNetV2`:

In [50]:
from torchvision.models import MobileNetV2

mobilenet = MobileNetV2(
    num_classes=10,
    width_mult=0.4,
    inverted_residual_setting=[
        # t, c, n, s
        [1, 16, 1, 1],
        [3, 24, 2, 2],
        [3, 32, 3, 2],
    ],
    dropout=0.2,
)

sum([param.numel() for param in mobilenet.parameters()])

46322

Но кроме архитектуры модели, мы также можем скачать веса, полученные при обучении на каком-то датасете. Например, для нашей задачи можно использовать предобучение на самом известном датасете для классификации изображений - ImageNet:

In [51]:
from torchvision.models.efficientnet import EfficientNet_B0_Weights, efficientnet_b0

# создаём EfficientNet с весами, полученными на ImageNet
weights = EfficientNet_B0_Weights.IMAGENET1K_V1
efficientnet = efficientnet_b0(weights=weights)
sum([param.numel() for param in efficientnet.parameters()])

5288548

**Указание 1.** С использованием модели в исходном виде есть проблема: в ImageNet 1000 классов, а у нас только 10. Поэтому в предобученной модели нужно будет полностью заменить последний линейный слой, который даёт распределение вероятностей классов. Это можно сделать уже в готовом объекте модели, переназначив атрибут.

Подсказка: в `efficientnet_b0` линейный слой находится в атрибуте `classifier`


**Указание 2.** Все слои, кроме нескольких последних (может быть, только последнего) мы можем заморозить, то есть сделать значения параметров в них неизменными. Это позволит и сохранить способность модели выделять полезные низкоуровневые признаки (она научилась этому на ImageNet), и существенно ускорить дообучение.


Чтобы заморозить параметры, нужно всего лишь отключить для них расчёт градиентов. Вернитесь к первой практике, чтобы вспомнить, как это можно сделать. Нам подойдёт самый простой способ с `.requires_grad`.

Подсказка: в `efficientnet_b0` свёрточные слои находятся в атрибуте `features`

**Указание 3.** Предобученные модели на ImageNet ожидают специальным образом трансформированные изображения:


In [52]:
weights.transforms()

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)

Поэтому эти трансформации нужно будет передать в датамодуль (как мы делали с аугментациями).

ВАШ ХОД: Обучите модель и выведите результат метода validate на удачном чекпоинте

In [55]:
class Datamodule2(L.LightningDataModule):
    def __init__(
        self,
        batch_size: int,
        transform: Callable[[Image], Tensor] = transforms.ToTensor(),
        num_workers: int = 0,
    ):
        super().__init__()
        self.batch_size = batch_size
        self.transform = transform
        self.num_workers = num_workers

    def prepare_data(self) -> None:
        # в этом методе можно сделать предварительную работу, например
        # скачать данные, сделать тяжёлый препроцессинг
        pass

    def setup(self, stage: str) -> None:
        # аргумент `stage` будет приходить из модуля обучения Trainer
        # на стадии обучения (fit) нам нужны оба датасета
        if stage == "fit":
            self.train_dataset = datasets.CIFAR10(
                "data",
                train=True,
                download=True,
                transform=self.transform,
            )
            self.val_dataset = datasets.CIFAR10(
                "data",
                train=False,
                download=True,
                transform=self.transform,
            )
        # на стадии валидации (validate) - только тестовый
        elif stage == "validate":
            self.val_dataset = datasets.CIFAR10(
                "data",
                train=False,
                download=True,
                transform=self.transform,
            )
        else:
            raise NotImplementedError
        # есть ещё стадии `test` и `predict`, но они нам не понадобятся

    def train_dataloader(self) -> TRAIN_DATALOADERS:
        return DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.num_workers,
        )

    def val_dataloader(self) -> EVAL_DATALOADERS:
        return DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.num_workers,
        )

In [62]:
efficientnet.classifier = nn.Sequential(nn.Dropout(p=0.2, inplace=True), nn.Linear(in_features=1280, out_features=10, bias=True))
for param in efficientnet.features[:-2:].parameters(): param.requires_grad = False

In [63]:
datamodule1 = Datamodule2(batch_size=256, num_workers=0, transform = weights.transforms())
datamodule1.setup(stage="fit")

trainer = L.Trainer(
    accelerator="auto",
    max_epochs=1,
    limit_train_batches=100,
    limit_val_batches=100,
    logger=TensorBoardLogger(save_dir="."),
)
lit_module = Lit(
    model= efficientnet,
    learning_rate=0.0007952812,
)
trainer.fit(
    model=lit_module,
    datamodule=datamodule1
)

Files already downloaded and verified
Files already downloaded and verified


INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | EfficientNet     | 4.0 M  | eval 
1 | train_metrics | MetricCollection | 0      | train
2 | val_metrics   | MetricCollection | 0      | train
-----------------------------------------------------------
1.1 M     Trainable params
2.9 M     Non-trainable params
4.0 M     Total params
16.081    Total estimated model params size (MB)
9         Modules in train mode
334       Modules in eval mode
INFO:lightning.pytorch.callbacks.model_summary:
  | Name          | Type             | Params | Mode 
-----------------------------------------------------------
0 | model         | EfficientNet     | 4.0 M  | eval 
1 | train_metrics | MetricCollection | 0      | train
2 | val_metrics   | MetricCollection | 0      | train
---------

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Validation Loss: 2.3447, Validation Accuracy: 0.0918


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1` reached.


Validation Loss: 0.2492, Validation Accuracy: 0.9176
